In [15]:
%matplotlib inline

In [16]:
# some code was made or modify with 
# OpenAI. (2023). ChatGPT (de ChatGPT del 3 de Agosto) [Large language model]. https://chat.openai.com

In [17]:
import os
import astropy.io.fits as py
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [18]:
plt.rcParams['figure.figsize'] = (20, 10)
plt.rcParams['font.size'] = 25

In [19]:
ls

Catalog/
EU Exoplanet population Interactive-F_XUV.ipynb
EU Exoplanet population Interactive-Selecting.ipynb
EU Exoplanet population Interactive.ipynb
EU finally_selected_targets.csv
NASA Exoplanet population Interactive-F_XUV.ipynb
NASA Exoplanet population Interactive-Selecting.ipynb
NASA Exoplanet population Interactive.ipynb
Pre Selection Candidates 2023b.numbers*


Data from https://exoplanetarchive.ipac.caltech.edu/docs/data.html

In [20]:
import plotly.express as px
import pandas as pd

In [21]:
# Load the CSV file into a DataFrame
file_path = 'Catalog/PSCompPars_2023.09.21_07.43.13.csv'  # Replace with your actual file path
df = pd.read_csv(file_path, comment='#')

In [22]:
# Filter out rows where the necessary parameters for TSM calculation are NaN
#filtered_df = df.dropna(subset=['radius', 'temp_calculated', 'mag_j', 'mass', 'orbital_period'])
#filtered_df = df.dropna(subset=['radius', 'temp_calculated', 'mag_v', 'mass', 'orbital_period'])
filtered_df = df.dropna(subset=['pl_radj', 'pl_eqt', 'sy_vmag', 'pl_bmassj', 'pl_orbper'])

In [23]:
# Constants
R_earth = 1.0  # Earth radius in Earth radii
T_earth = 288.0  # Earth's temperature in Kelvin
V_0 = 9  # Reference V-band magnitude

---
## Transmission Spectroscopy Metric (TSM) 
---

$$\boxed{\text{TSM} = \left( \frac{R_p}{R_{\oplus}} \right)^3 \times \left( \frac{T_{\text{eq}}}{T_{\oplus}} \right) \times 10^{-0.4 \times (V - V_0)}}$$

In [24]:
# Calculate TSM
filtered_df['TSM'] = (filtered_df['pl_rade'] / R_earth) ** 3 * (filtered_df['pl_eqt'] / T_earth) * 10 ** (-0.4 * (filtered_df['sy_vmag'] - V_0))
TSM = filtered_df['TSM']

/var/folders/rb/x7czqcn50038tc9lzxkw7z4h0000gn/T/ipykernel_4818/2671460340.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



---

In [25]:
filtered_df = df.dropna(subset=['pl_radj', 'pl_bmassj', 'pl_orbper'])

# Calculate TSM
filtered_df['TSM'] = (filtered_df['pl_rade'] / R_earth) ** 3 * (filtered_df['pl_eqt'] / T_earth) * 10 ** (-0.4 * (filtered_df['sy_vmag'] - V_0))
# Apply log10 transformation to the 'TSM' column
filtered_df['log_TSM'] = np.log10(filtered_df['TSM'])

/var/folders/rb/x7czqcn50038tc9lzxkw7z4h0000gn/T/ipykernel_4818/4203060872.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/rb/x7czqcn50038tc9lzxkw7z4h0000gn/T/ipykernel_4818/4203060872.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [26]:
# Create the interactive plot
# Calculate the size array for the markers
default_size = 16  # Default size for null values
size_array = 17 - filtered_df['sy_vmag'].fillna(default_size)
# Convert all values below 1 to 1
size_array = np.where(size_array < 1, 1, size_array)

# Your scatter plot using log-transformed 'TSM' for coloring
fig = px.scatter(filtered_df, x='pl_orbper', y='pl_bmassj', color='log_TSM', hover_data=['pl_name'], size=size_array)
fig.update_xaxes(type='log', range=[np.log10(10**-1), np.log10(10**1.5)])  # Logarithmic range for x
fig.update_yaxes(type='log', range=[np.log10(10**-3), np.log10(10**1.5)])  # Logarithmic range for y


# Your lines data on linear scale
x = np.linspace(10**-1, 5, 100)  # Corresponding to original np.linspace(-1, np.log10(5))
x2 = np.linspace(10**-1, 10, 100)  # Corresponding to original np.linspace(-1, np.log10(10))
# Mazeh, T., Holczer, T., & Faigler, S. (2016). 
# Dearth of short-period Neptunian exoplanets - a desert in period-mass and period-radius planes. 
# Astronomy & Astrophysics, 589, A75. https://dx.doi.org/10.1051/0004-6361/201528065
# Calculate y-values based on x-values
y1 = x**(-1.14) * 10**0.23
y2 = x2**(-1.14) * 10**0.23
y3 = x**(0.98) * 10**(-1.85)
y4 = x2**(0.98) * 10**(-1.85)


# Add lines to the existing figure
import plotly.graph_objects as go
fig.add_trace(go.Scatter(x=x, y=y1, mode='lines', line=dict(color='black', dash='dash', width=3), showlegend=False))
fig.add_trace(go.Scatter(x=x2, y=y2, mode='lines', line=dict(color='black', dash='dash', width=1), showlegend=False))
fig.add_trace(go.Scatter(x=x, y=y3, mode='lines', line=dict(color='black', dash='dash', width=3), showlegend=False))
fig.add_trace(go.Scatter(x=x2, y=y4, mode='lines', line=dict(color='black', dash='dash', width=1), showlegend=False))


# Filter the DataFrame to include only the specified names
star_names = ['WASP-69 b', 'WASP-52 b', 'HAT-P-18 b', 'WASP-80 b', 'WASP-177 b', 'HAT-P-26 b', 'NGTS-5 b', 'LTT 9779 b']
star_df = filtered_df[filtered_df['pl_name'].isin(star_names)]

# Add a scatter trace with star markers for these points
fig.add_trace(go.Scatter(
    x=star_df['pl_orbper'],
    y=star_df['pl_bmassj'],
    mode='markers',
    marker=dict(
        symbol='star',  # Set marker shape as star
        size=13,        # Marker size
        color='green'     # Marker color
    ),
    customdata=np.stack((star_df['pl_name'], star_df['pl_bmassj'], star_df['pl_orbper'], star_df['log_TSM']), axis=1),
    hovertemplate=
        "Star: %{customdata[0]}<br>" +
        "Mass: %{customdata[1]} Jupiter Masses<br>" +
        "Orbital Period: %{customdata[2]} days<br>" +
        "TSM: %{customdata[3]}",
    showlegend=False  # Remove this trace from the legend
))



# Update layout for axis labels and title
fig.update_layout(
    xaxis_title='Orbital Period (days)',
    yaxis_title='Mass (Jupiter Masses)',
    title={
        'text': 'Orbital Period vs Mass of Exoplanets<br>Colored by TSM',
        'x': 0.5,  # Center the title
        'xanchor': 'center',
        'yanchor': 'top'
    },
    font=dict(
        size=14  # This will set the font size for both axis labels
    ),
    title_font=dict(
        size=16  # This sets the font size for the title
    ),
    width=1000,  # Width in pixels
    height=800   # Height in pixels
)

# Filtrando por TSM

In [33]:
# Create the interactive plot
# Filter the DataFrame to only include rows with log_TSM > 2.5
filtered_df_cut = filtered_df[filtered_df['log_TSM'] > 2.5]
# Calculate the size array for the markers
default_size = 16  # Default size for null values
size_array = 17 - filtered_df_cut['sy_vmag'].fillna(default_size)
# Convert all values below 1 to 1
size_array = np.where(size_array < 1, 1, size_array)

# Your scatter plot using log-transformed 'TSM' for coloring
fig = px.scatter(filtered_df_cut, x='pl_orbper', y='pl_bmassj', color='log_TSM', hover_data=['pl_name'], size=size_array)
fig.update_xaxes(type='log', range=[np.log10(10**-1), np.log10(10**1.5)])  # Logarithmic range for x
fig.update_yaxes(type='log', range=[np.log10(10**-3), np.log10(10**1.5)])  # Logarithmic range for y


# Your lines data on linear scale
x = np.linspace(10**-1, 5, 100)  # Corresponding to original np.linspace(-1, np.log10(5))
x2 = np.linspace(10**-1, 10, 100)  # Corresponding to original np.linspace(-1, np.log10(10))
# Mazeh, T., Holczer, T., & Faigler, S. (2016). 
# Dearth of short-period Neptunian exoplanets - a desert in period-mass and period-radius planes. 
# Astronomy & Astrophysics, 589, A75. https://dx.doi.org/10.1051/0004-6361/201528065
# Calculate y-values based on x-values
y1 = x**(-1.14) * 10**0.23
y2 = x2**(-1.14) * 10**0.23
y3 = x**(0.98) * 10**(-1.85)
y4 = x2**(0.98) * 10**(-1.85)


# Add lines to the existing figure
import plotly.graph_objects as go
fig.add_trace(go.Scatter(x=x, y=y1, mode='lines', line=dict(color='black', dash='dash', width=3), showlegend=False))
fig.add_trace(go.Scatter(x=x2, y=y2, mode='lines', line=dict(color='black', dash='dash', width=1), showlegend=False))
fig.add_trace(go.Scatter(x=x, y=y3, mode='lines', line=dict(color='black', dash='dash', width=3), showlegend=False))
fig.add_trace(go.Scatter(x=x2, y=y4, mode='lines', line=dict(color='black', dash='dash', width=1), showlegend=False))


# Filter the DataFrame to include only the specified names
star_names = ['WASP-69 b', 'WASP-52 b', 'HAT-P-18 b', 'WASP-80 b', 'WASP-177 b', 'HAT-P-26 b', 'NGTS-5 b', 'LTT 9779 b']
star_df = filtered_df[filtered_df['pl_name'].isin(star_names)]

# Add a scatter trace with star markers for these points
fig.add_trace(go.Scatter(
    x=star_df['pl_orbper'],
    y=star_df['pl_bmassj'],
    mode='markers',
    marker=dict(
        symbol='star',  # Set marker shape as star
        size=13,        # Marker size
        color='green'     # Marker color
    ),
    customdata=np.stack((star_df['pl_name'], star_df['pl_bmassj'], star_df['pl_orbper'], star_df['log_TSM']), axis=1),
    hovertemplate=
        "Star: %{customdata[0]}<br>" +
        "Mass: %{customdata[1]} Jupiter Masses<br>" +
        "Orbital Period: %{customdata[2]} days<br>" +
        "TSM: %{customdata[3]}",
    showlegend=False  # Remove this trace from the legend
))



# Update layout for axis labels and title
fig.update_layout(
    xaxis_title='Orbital Period (days)',
    yaxis_title='Mass (Jupiter Masses)',
    title={
        'text': 'Orbital Period vs Mass of Exoplanets<br>Colored by TSM',
        'x': 0.5,  # Center the title
        'xanchor': 'center',
        'yanchor': 'top'
    },
    font=dict(
        size=14  # This will set the font size for both axis labels
    ),
    title_font=dict(
        size=16  # This sets the font size for the title
    ),
    width=1000,  # Width in pixels
    height=800   # Height in pixels
)

# Filtrando por dec < 10

In [35]:
# Create the interactive plot
# Filter the DataFrame to only include dec < 10
filtered_df_cut = filtered_df_cut[filtered_df_cut['dec'] < 10]
# Calculate the size array for the markers
default_size = 16  # Default size for null values
size_array = 17 - filtered_df_cut['sy_vmag'].fillna(default_size)
# Convert all values below 1 to 1
size_array = np.where(size_array < 1, 1, size_array)

# Your scatter plot using log-transformed 'TSM' for coloring
fig = px.scatter(filtered_df_cut, x='pl_orbper', y='pl_bmassj', color='log_TSM', hover_data=['pl_name'], size=size_array)
fig.update_xaxes(type='log', range=[np.log10(10**-1), np.log10(10**1.5)])  # Logarithmic range for x
fig.update_yaxes(type='log', range=[np.log10(10**-3), np.log10(10**1.5)])  # Logarithmic range for y


# Your lines data on linear scale
x = np.linspace(10**-1, 5, 100)  # Corresponding to original np.linspace(-1, np.log10(5))
x2 = np.linspace(10**-1, 10, 100)  # Corresponding to original np.linspace(-1, np.log10(10))
# Mazeh, T., Holczer, T., & Faigler, S. (2016). 
# Dearth of short-period Neptunian exoplanets - a desert in period-mass and period-radius planes. 
# Astronomy & Astrophysics, 589, A75. https://dx.doi.org/10.1051/0004-6361/201528065
# Calculate y-values based on x-values
y1 = x**(-1.14) * 10**0.23
y2 = x2**(-1.14) * 10**0.23
y3 = x**(0.98) * 10**(-1.85)
y4 = x2**(0.98) * 10**(-1.85)


# Add lines to the existing figure
import plotly.graph_objects as go
fig.add_trace(go.Scatter(x=x, y=y1, mode='lines', line=dict(color='black', dash='dash', width=3), showlegend=False))
fig.add_trace(go.Scatter(x=x2, y=y2, mode='lines', line=dict(color='black', dash='dash', width=1), showlegend=False))
fig.add_trace(go.Scatter(x=x, y=y3, mode='lines', line=dict(color='black', dash='dash', width=3), showlegend=False))
fig.add_trace(go.Scatter(x=x2, y=y4, mode='lines', line=dict(color='black', dash='dash', width=1), showlegend=False))


# Filter the DataFrame to include only the specified names
star_names = ['WASP-69 b', 'WASP-52 b', 'HAT-P-18 b', 'WASP-80 b', 'WASP-177 b', 'HAT-P-26 b', 'NGTS-5 b', 'LTT 9779 b']
star_df = filtered_df[filtered_df['pl_name'].isin(star_names)]

# Add a scatter trace with star markers for these points
fig.add_trace(go.Scatter(
    x=star_df['pl_orbper'],
    y=star_df['pl_bmassj'],
    mode='markers',
    marker=dict(
        symbol='star',  # Set marker shape as star
        size=13,        # Marker size
        color='green'     # Marker color
    ),
    customdata=np.stack((star_df['pl_name'], star_df['pl_bmassj'], star_df['pl_orbper'], star_df['log_TSM']), axis=1),
    hovertemplate=
        "Star: %{customdata[0]}<br>" +
        "Mass: %{customdata[1]} Jupiter Masses<br>" +
        "Orbital Period: %{customdata[2]} days<br>" +
        "TSM: %{customdata[3]}",
    showlegend=False  # Remove this trace from the legend
))



# Update layout for axis labels and title
fig.update_layout(
    xaxis_title='Orbital Period (days)',
    yaxis_title='Mass (Jupiter Masses)',
    title={
        'text': 'Orbital Period vs Mass of Exoplanets<br>Colored by TSM',
        'x': 0.5,  # Center the title
        'xanchor': 'center',
        'yanchor': 'top'
    },
    font=dict(
        size=14  # This will set the font size for both axis labels
    ),
    title_font=dict(
        size=16  # This sets the font size for the title
    ),
    width=1000,  # Width in pixels
    height=800   # Height in pixels
)

# Filtrando por mag_v < 13

In [45]:
# Create the interactive plot
# Filter the DataFrame to only include mag_v < 13
filtered_df_cut = filtered_df_cut[filtered_df_cut['sy_vmag'] < 13]
# Calculate the size array for the markers
default_size = 16  # Default size for null values
size_array = 17 - filtered_df_cut['sy_vmag'].fillna(default_size)
# Convert all values below 1 to 1
size_array = np.where(size_array < 1, 1, size_array)

# Your scatter plot using log-transformed 'TSM' for coloring
fig = px.scatter(filtered_df_cut, x='pl_orbper', y='pl_bmassj', color='log_TSM', hover_data=['pl_name'], 
                 size=size_array, range_color=[2.5, 4])
fig.update_xaxes(type='log', range=[np.log10(0.6), np.log10(9)])  # Logarithmic range for x
fig.update_yaxes(type='log', range=[np.log10(0.09), np.log10(3)])  # Logarithmic range for y


# Your lines data on linear scale
x = np.linspace(10**-1, 5, 100)  # Corresponding to original np.linspace(-1, np.log10(5))
x2 = np.linspace(10**-1, 10, 100)  # Corresponding to original np.linspace(-1, np.log10(10))
# Mazeh, T., Holczer, T., & Faigler, S. (2016). 
# Dearth of short-period Neptunian exoplanets - a desert in period-mass and period-radius planes. 
# Astronomy & Astrophysics, 589, A75. https://dx.doi.org/10.1051/0004-6361/201528065
# Calculate y-values based on x-values
y1 = x**(-1.14) * 10**0.23
y2 = x2**(-1.14) * 10**0.23
y3 = x**(0.98) * 10**(-1.85)
y4 = x2**(0.98) * 10**(-1.85)


# Add lines to the existing figure
import plotly.graph_objects as go
fig.add_trace(go.Scatter(x=x, y=y1, mode='lines', line=dict(color='black', dash='dash', width=3), showlegend=False))
fig.add_trace(go.Scatter(x=x2, y=y2, mode='lines', line=dict(color='black', dash='dash', width=1), showlegend=False))
fig.add_trace(go.Scatter(x=x, y=y3, mode='lines', line=dict(color='black', dash='dash', width=3), showlegend=False))
fig.add_trace(go.Scatter(x=x2, y=y4, mode='lines', line=dict(color='black', dash='dash', width=1), showlegend=False))


# Filter the DataFrame to include only the specified names
star_names = ['WASP-69 b', 'WASP-52 b', 'HAT-P-18 b', 'WASP-80 b', 'WASP-177 b', 'HAT-P-26 b', 'NGTS-5 b', 'LTT 9779 b']
star_df = filtered_df[filtered_df['pl_name'].isin(star_names)]

# Add a scatter trace with star markers for these points
fig.add_trace(go.Scatter(
    x=star_df['pl_orbper'],
    y=star_df['pl_bmassj'],
    mode='markers',
    marker=dict(
        symbol='star',  # Set marker shape as star
        size=13,        # Marker size
        color='green'     # Marker color
    ),
    customdata=np.stack((star_df['pl_name'], star_df['pl_bmassj'], star_df['pl_orbper'], star_df['log_TSM']), axis=1),
    hovertemplate=
        "Star: %{customdata[0]}<br>" +
        "Mass: %{customdata[1]} Jupiter Masses<br>" +
        "Orbital Period: %{customdata[2]} days<br>" +
        "TSM: %{customdata[3]}",
    showlegend=False  # Remove this trace from the legend
))



# Update layout for axis labels and title
fig.update_layout(
    xaxis_title='Orbital Period (days)',
    yaxis_title='Mass (Jupiter Masses)',
    title={
        'text': 'Orbital Period vs Mass of Exoplanets<br>Colored by TSM',
        'x': 0.5,  # Center the title
        'xanchor': 'center',
        'yanchor': 'top'
    },
    font=dict(
        size=14  # This will set the font size for both axis labels
    ),
    title_font=dict(
        size=16  # This sets the font size for the title
    ),
    width=1000,  # Width in pixels
    height=800   # Height in pixels
)

In [47]:
#pre_selected = ("WASP-19 b", "WASP-103 b", "WASP-12 b", "TOI-1937 b", "Qatar-10 b", "WASP-76 b", "WASP-92 b",
#            "HAT-P-65 b", "WASP-126 b", "HATS-21 b", "Qatar-8 b", "WASP-69 b", "WASP-181 b", "TOI-954 b",
#            "KELT-11 b", "WASP-131 b", "WASP-20 A b", "HAT-P-67 b", "WASP-193 b", "WASP-166 b")

In [48]:
pre_selected = ("WASP-19 b", "WASP-121 b", "WASP-4 b", "WASP-145 A b", "WASP-49 b", "WASP-6 b",
            "TOI-2842 b", "WASP-126 b", "WASP-174 b", "TOI-2421 b", "TOI-954 b", "WASP-147 b",
            "Wasp-20 b", "Wasp-131 b")

In [50]:
pre_selected_df = filtered_df_cut[filtered_df_cut['pl_name'].isin(pre_selected)]

# Create the interactive plot
# Filter the DataFrame to only include mag_v < 13
filtered_df_cut = filtered_df_cut[filtered_df_cut['sy_vmag'] < 13]
# Calculate the size array for the markers
default_size = 16  # Default size for null values
size_array = 17 - filtered_df_cut['sy_vmag'].fillna(default_size)
# Convert all values below 1 to 1
size_array = np.where(size_array < 1, 1, size_array)

# Your scatter plot using log-transformed 'TSM' for coloring
fig = px.scatter(filtered_df_cut, x='pl_orbper', y='pl_bmassj', color='log_TSM', hover_data=['pl_name'], 
                 size=size_array, range_color=[2.5, 4])
fig.update_xaxes(type='log', range=[np.log10(0.6), np.log10(9)])  # Logarithmic range for x
fig.update_yaxes(type='log', range=[np.log10(0.09), np.log10(3)])  # Logarithmic range for y


# Your lines data on linear scale
x = np.linspace(10**-1, 5, 100)  # Corresponding to original np.linspace(-1, np.log10(5))
x2 = np.linspace(10**-1, 10, 100)  # Corresponding to original np.linspace(-1, np.log10(10))
# Mazeh, T., Holczer, T., & Faigler, S. (2016). 
# Dearth of short-period Neptunian exoplanets - a desert in period-mass and period-radius planes. 
# Astronomy & Astrophysics, 589, A75. https://dx.doi.org/10.1051/0004-6361/201528065
# Calculate y-values based on x-values
y1 = x**(-1.14) * 10**0.23
y2 = x2**(-1.14) * 10**0.23
y3 = x**(0.98) * 10**(-1.85)
y4 = x2**(0.98) * 10**(-1.85)


# Add lines to the existing figure
import plotly.graph_objects as go
fig.add_trace(go.Scatter(x=x, y=y1, mode='lines', line=dict(color='black', dash='dash', width=3), showlegend=False))
fig.add_trace(go.Scatter(x=x2, y=y2, mode='lines', line=dict(color='black', dash='dash', width=1), showlegend=False))
fig.add_trace(go.Scatter(x=x, y=y3, mode='lines', line=dict(color='black', dash='dash', width=3), showlegend=False))
fig.add_trace(go.Scatter(x=x2, y=y4, mode='lines', line=dict(color='black', dash='dash', width=1), showlegend=False))


# Filter the DataFrame to include only the specified names
star_names = ['WASP-69 b', 'WASP-52 b', 'HAT-P-18 b', 'WASP-80 b', 'WASP-177 b', 'HAT-P-26 b', 'NGTS-5 b', 'LTT 9779 b']
star_df = filtered_df[filtered_df['pl_name'].isin(star_names)]

# Ahora añadimos un nuevo trazo para estos planetas seleccionados con una estrella como marcador
fig.add_trace(go.Scatter(
    x=pre_selected_df['pl_orbper'],
    y=pre_selected_df['pl_bmassj'],
    mode='markers',
    marker=dict(
        symbol='star',  # Establecemos la forma del marcador como estrella
        size=13,        # Tamaño del marcador
        color='black'     # Color del marcador
    ),
    customdata=np.stack((pre_selected_df['pl_name'], pre_selected_df['pl_bmassj'], pre_selected_df['pl_orbper'], pre_selected_df['log_TSM']), axis=1),
    hovertemplate=
        "Star: %{customdata[0]}<br>" +
        "Mass: %{customdata[1]} Jupiter Masses<br>" +
        "Orbital Period: %{customdata[2]} days<br>" +
        "TSM: %{customdata[3]}",
    showlegend=False  # Removemos este trazo del leyenda
))



# Update layout for axis labels and title
fig.update_layout(
    xaxis_title='Orbital Period (days)',
    yaxis_title='Mass (Jupiter Masses)',
    title={
        'text': 'Orbital Period vs Mass of Exoplanets<br>Colored by TSM',
        'x': 0.5,  # Center the title
        'xanchor': 'center',
        'yanchor': 'top'
    },
    font=dict(
        size=14  # This will set the font size for both axis labels
    ),
    title_font=dict(
        size=16  # This sets the font size for the title
    ),
    width=1000,  # Width in pixels
    height=800   # Height in pixels
)

In [52]:
finelly_selected = pre_selected
# Assuming the column containing the names of the targets is called 'name'
filtered_df = df[df['pl_name'].isin(finelly_selected)]

filtered_df.to_csv('Nasa_finally_selected_targets.csv', index=False)